In [39]:
import pandas as pd
import numpy as np
products = pd.read_csv('amazon_baby.csv')
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [19]:
len(products) - len(products.dropna())

1147

In [20]:
len(products)

183531

In [21]:
products = products.dropna()

In [22]:
for i in range(0,len(products)-1):
    if type(products.iloc[i]['review']) != str:
        products.iloc[i]['review'] = str(products.iloc[i]['review'])

In [23]:
products = products[products['rating'] != 3]

In [24]:
def sentiment(n):
    return 1 if n >= 4 else 0
products['sentiment'] = products['rating'].apply(sentiment)
products.head()

,name,review,rating,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,1


In [25]:
def combined_features(row):
    return row['name'] + ' '+ row['review']
products['all_features'] = products.apply(combined_features, axis=1)
products.head()

,name,review,rating,sentiment,all_features
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,1,Planetwise Wipe Pouch it came early and was no...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,1,Annas Dream Full Quilt with 2 Shams Very soft ...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,1,Stop Pacifier Sucking without tears with Thumb...
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,Stop Pacifier Sucking without tears with Thumb...
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,1,Stop Pacifier Sucking without tears with Thumb...


In [26]:
def remove_punctuation(text):
    import string
    return text.translate(str.maketrans('','',string.punctuation)) 

products['review_clean'] = products['review'].apply(remove_punctuation)
products = products[products['rating'] != 3]

In [27]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [28]:
X = products['all_features']
y = products['sentiment']

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
ctmTr = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression()
model.fit(ctmTr, y_train)

C:\Users\Rachitha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [43]:
sample_test_matrix = cv.transform(X_test['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

KeyError: 'review_clean'

In [40]:
for i in range(len(X_test)):
    prob = 1/(1+np.exp(-scores[i]))
    print('The probability that a sentiment is positive for observation',i+1, 'is:', prob)

NameError: name 'scores' is not defined

In [14]:
y_pred_class = model.predict(X_test_dtm)
print(y_pred_class)

[1 1 1 ... 1 1 0]


In [15]:
accuracy_score(y_test, y_pred_class)

0.9321100917431193